In [17]:
!pip3 install openai langchain -q
!pip3 install langchain_openai
!pip3 install faiss-cpu
!pip3 install langchain_community
!pip3 install unstructured
!pip3 install  unstructured[all-docs]
#https://python.langchain.com/docs/integrations/document_loaders/unstructured_file





zsh:1: no matches found: unstructured[all-docs]


In [18]:
import dotenv
import os

dotenv.load_dotenv() 

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [19]:
from glob import glob

files = glob("*")

In [20]:
from langchain_community.document_loaders import DirectoryLoader
loader = DirectoryLoader('/users/luis/Documents/obsvault/', glob="**/*.md")
docs = loader.load()
len(docs)

8

In [21]:
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(docs)
embeddings = OpenAIEmbeddings()
db = FAISS.from_documents(texts, embeddings)

In [22]:
retriever = db.as_retriever(search_kwargs={"k": 3})

In [24]:
from langchain import OpenAI
from langchain.chains import RetrievalQA
from langchain.agents import Tool

openai_llm = OpenAI(
    openai_api_key=OPENAI_API_KEY,
    temperature=0
)
qa_llm_chain = RetrievalQA.from_chain_type(llm=openai_llm,  retriever=retriever)
langchain_tool = Tool(
    name='LangChain',
    func=qa_llm_chain.run,
    description='You must use this tool to answer questions about LangChain'
)

ValidationError: 1 validation error for RetrievalQA
retriever
  Can't instantiate abstract class BaseRetriever with abstract method _get_relevant_documents (type=type_error)

In [ ]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

prompt = PromptTemplate(
    input_variables=["query"],
    template="{query}"
)
llm_chain = LLMChain(llm=openai_llm, prompt=prompt)
llm_tool = Tool(
    name='Language Model',
    func=llm_chain.run,
    description='Use this tool for general purpose queries not about LangChain'
)

In [ ]:
from langchain.agents import AgentType, initialize_agent

agent = initialize_agent(
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    tools=[langchain_tool, llm_tool],
    llm=openai_llm,
    verbose=True,
)

NameError: name 'langchain_tool' is not defined